v1 meta store product availability checker

In [ ]:
##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd


# get text and game name
# $future task: enable chrome headless mode, per https://stackoverflow.com/questions/46920243/how-to-configure-chromedriver-to-initiate-chrome-browser-in-headless-mode-throug/49582462#49582462
############
print('What game are you searching for? ')
# game_title = input().lower()
game_title = 'grand theft auto'
# game_title = 'Baldur\'s Gate 3'.lower()
print(game_title)
print()
url_game_title = game_title.replace(' ', '%20').replace('\'', '%27').title()
url = 'https://www.cdkeys.com/?q=' + url_game_title

driver = webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

# capture region lock, game system, product type, price
pattern = re.escape(game_title) + r"(.*?)" + re.escape('add to wish list')
matches = re.findall(pattern, text)

# clean matches by price
price_clean = []
for i in matches:
    if 'out of stock' in i:
        oos_match = i.replace('out of stock', '`out of stock')
        price_clean.append(oos_match)
    if '$' in i:
        instock_match = i.replace('$', '`$')
        price_clean.append(instock_match)

# clean matches by platform
platform_clean = []
for i in price_clean:
    if '(' in i:
        platform_match = i.replace('(', '`(')
        platform_clean.append(platform_match)

# clean matches by platform, pt2
platform2_clean = []
for i in platform_clean:
    if 'xbox one & xbox series x|s' in i:
        xbox_windows_match = i.replace('xbox one & xbox series x|s', 'xbox one / series x|s')
        platform2_clean.append(xbox_windows_match)

    else:
        platform2_clean.append(i)

# clean matches by region
region_clean = []
for i in platform2_clean:
    if ')' in i:
        region_match = i.replace(')', ')`')
        region_clean.append(region_match)

# clean matches by type
type_clean = []
for i in region_clean:
    if 'xbox' in i:
        type_match = i.replace('xbox', '`xbox')
        type_clean.append(type_match)

df = pd.DataFrame(type_clean) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df1 = df1.drop(columns=[3])
df1.columns = ['product', 'platform', 'region', 'price', 'original price'] # add column titles
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['product'] = df1['product'].str.replace(': ', '')
df1['region'] = df1['region'].str.replace('(', '').str.replace(')', '').str.replace(')      rating:   %', '')
df1['price'] = df1['price'].str.replace('$', '')
df1['original price'] = df1['original price'].str.replace('$', '')
df1['site'] = 'CDKeys'

print('CDKeys\' <<' + game_title + '>>')
cdkeys = df1.sort_values(by=['platform', 'price'])
display(cdkeys) 

In [26]:
##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

cleaned = []

product_list = ['quest 3 standard facial interface',
                'quest 3 elite strap battery', 'quest 3 carrying case']

for product in product_list:
    url_completion = product.replace(' ', '-')

    url = 'https://www.meta.com/quest/accessories/{}'.format(url_completion)

    driver = webdriver.Chrome()  # bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    text = soup.get_text().lower().replace('\n', '')

    if product == 'quest 3 elite strap battery':  # account for different product names in metadata
        product_pattern = 'quest 3 elite strap'
    else:
        product_pattern = product

    pattern = '{} (.*?) usd'.format(re.escape(product_pattern))
    match = re.search(pattern, text)
    match_text = match.group()

    # column separator prep for pandas table
    replacement_key = product.split()[-1]

    if '$' in match_text:
        cleaned.append(match_text.replace('$', '`$').replace(
            replacement_key, replacement_key + '`'))

print(cleaned)

['quest 3 standard facial interface` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaq  add to bag`$34.99 usd', 'quest 3 elite strap with battery` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaqreviews  notify me`$129.99 usd', 'quest 3 carrying case` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaqreviews  add to bag`$69.99 usd']
